# Summary:
- No SMOTE
- Based on the 32x32x3 vectors
- 1D Cov + two layers of LSTM is tested
- Validation set accuracy =  0.8576810380663019

In [1]:
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE, ADASYN

import numpy as np
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
np.random.seed(1)

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


# 1. Data Preparation

In [2]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [4]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [5]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [6]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((14333, 3072), (14333,), (4778, 3072), (4778,), 19111)

In [7]:
xtr, xt = X_train_array.shape, X_test_array.shape

In [8]:
# reshape dataset
X_train = X_train_array.reshape(xtr[0],96,32) 
Y_train = np.eye(2)[Y_train_array.astype(int)] 
X_test = X_test_array.reshape(xt[0],96,32) 
Y_test = np.eye(2)[Y_test_array.astype(int)] 

# 2. LSTM

In [9]:
def model(input_shape):
    
    X_input = Input(shape = input_shape)
    
    # Cov layer
    X = Conv1D(filters=768, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(512, return_sequences=True)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(512)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = X_input, outputs = X)
        
    return model

In [10]:
model = model(input_shape = (96,32))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 96, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 21, 768)           369408    
_________________________________________________________________
batch_normalization_1 (Batch (None, 21, 768)           3072      
_________________________________________________________________
activation_1 (Activation)    (None, 21, 768)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 21, 768)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 21, 512)           2623488   
_________________________________________________________________
dropout_2 (Dropout)          (None, 21, 512)           0         
__________

In [12]:
from keras.utils import plot_model
plot_model(model, to_file='LSTMModel.png')

<img src='LSTMModel.png'>

In [13]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [14]:
model.fit(X_train, Y_train, batch_size = 64, epochs=200)

Epoch 1/200
14333/14333 [==============================] - 31s 2ms/step - loss: 0.4789 - acc: 0.7768
Epoch 2/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.3871 - acc: 0.8398
Epoch 3/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.3597 - acc: 0.8539
Epoch 4/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.3468 - acc: 0.8586
Epoch 5/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.3388 - acc: 0.8615
Epoch 6/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.3308 - acc: 0.8688
Epoch 7/200
14333/14333 [==============================] - 26s 2ms/step - loss: 0.3259 - acc: 0.8672
Epoch 8/200
14333/14333 [==============================] - 26s 2ms/step - loss: 0.3166 - acc: 0.8732
Epoch 9/200
14333/14333 [==============================] - 26s 2ms/step - loss: 0.3122 - acc: 0.8743
Epoch 10/200
14333/14333 [==============================] - 26s 2ms/step - loss: 0.3094 - a

14333/14333 [==============================] - 25s 2ms/step - loss: 0.2272 - acc: 0.9115
Epoch 82/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2265 - acc: 0.9113
Epoch 83/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2238 - acc: 0.9150
Epoch 84/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2202 - acc: 0.9155
Epoch 85/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2213 - acc: 0.9137
Epoch 86/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2209 - acc: 0.9134
Epoch 87/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2235 - acc: 0.9121
Epoch 88/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2226 - acc: 0.9134
Epoch 89/200
14333/14333 [==============================] - 26s 2ms/step - loss: 0.2222 - acc: 0.9147
Epoch 90/200
14333/14333 [==============================] - 26s 2ms/step - loss: 0.2239 - acc: 

14333/14333 [==============================] - 25s 2ms/step - loss: 0.2000 - acc: 0.9240
Epoch 161/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.1999 - acc: 0.9228
Epoch 162/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2001 - acc: 0.9205
Epoch 163/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2024 - acc: 0.9213
Epoch 164/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2036 - acc: 0.9211
Epoch 165/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2031 - acc: 0.9200
Epoch 166/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.1950 - acc: 0.9250
Epoch 167/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2053 - acc: 0.9214
Epoch 168/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.2035 - acc: 0.9225
Epoch 169/200
14333/14333 [==============================] - 25s 2ms/step - loss: 0.201

In [15]:
model_json = model.to_json()
with open ("LSTMModel.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("LSTMModel.h5")

In [16]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4778/4778 [==============================] - 4s 858us/step
Validation set accuracy =  0.8547509417917101


In [17]:
from sklearn.metrics import classification_report

pred = model.predict(X_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(Y_test, axis=1), predicted)
print(report)

4778/4778 [==============================] - 4s 817us/step
             precision    recall  f1-score   support

          0       0.90      0.84      0.87      2694
          1       0.81      0.88      0.84      2084

avg / total       0.86      0.85      0.86      4778

